In [2]:
from deepeval.synthesizer import Synthesizer
from deepeval.dataset import Golden
from deepeval.synthesizer.config import StylingConfig
from repositories.phone import get_all as get_all_phones


In [ ]:
phones = get_all_phones()
context_format = "Thông tin về câu hỏi và câu trả lời thường gặp của khách hàng tại FPT Shop:\nCâu hỏi: {question}\nCâu trả lời: {answer}\n"

phone_styling_config = StylingConfig(
    input_format="Các câu hỏi bằng tiếng Việt liên quan đến sản phẩm, chính sách bảo hành, thanh toán, giao hàng, và dịch vụ tại FPT Shop.",
    expected_output_format="Câu trả lời tư vấn rõ ràng, chính xác, thể hiện sự chuyên nghiệp và thân thiện như một nhân viên bán hàng của FPT Shop.",
    task="Trả lời các câu hỏi thường gặp của khách hàng về sản phẩm và dịch vụ của FPT Shop nhằm hỗ trợ bán hàng và chăm sóc khách hàng.",
    scenario="Khách hàng tiềm năng hoặc khách hàng hiện tại đang cần được giải đáp nhanh chóng về thông tin mua sắm, chính sách và dịch vụ tại FPT Shop.",
)

In [ ]:
class PhoneTestCaseType(str, Enum):
    NAME_ONLY = "name_only"          # Chỉ hỏi về tên điện thoại cụ thể
    BRAND_ONLY = "brand_only"        # Hỏi về điện thoại của một hãng
    PRICE_ONLY = "price_only"        # Hỏi về điện thoại trong khoảng giá
    BRAND_PRICE = "brand_price"      # Kết hợp hãng và giá
    BRAND_NAME = "brand_name"        # Kết hợp hãng và tên
    PRICE_NAME = "price_name"        # Kết hợp giá và tên
    ALL_FILTERS = "all_filters"      # Kết hợp cả 3 loại filter

In [ ]:
from deepeval.test_case import LLMTestCase
from service.phone import search
from service.phone import format_phones_to_context
from service.phone import PhoneFilter, Config
from repositories.phone import PhoneModel
import weave

@weave.op(name="generate_phone_test_case")
def generate_phone_test_cases(base_case: dict) -> list[LLMTestCase]:
    # Lấy kết quả thực từ hệ thống
    phones = search(base_case["filter"])
    
    # Tạo context từ kết quả phones
    context = format_phones_to_context(phones)
    
    # Sử dụng synthesizer để tạo các biến thể của câu hỏi
    synthesizer = Synthesizer(
        model="gpt-4o-mini",
        styling_config=PhoneStylingConfig(
            input_format="Các câu hỏi về điện thoại theo " + base_case["type"],
            expected_output_format="Câu trả lời chuyên nghiệp về thông tin điện thoại",
            task="Trả lời thông tin về điện thoại theo yêu cầu cụ thể",
            scenario="Khách hàng đang tìm kiếm điện thoại với các tiêu chí cụ thể"
        )
    )
    
    return synthesizer.generate_goldens_from_contexts([[context]])

In [ ]:
base_test_cases = [
    {
        "type": PhoneTestCaseType.NAME_ONLY,
        "input": "Cho tôi thông tin về iPhone 15 Pro Max",
        "filter": PhoneFilter(
            name="iPhone 15 Pro Max",
            config=Config(is_recommending=False)
        )
    },
    {
        "type": PhoneTestCaseType.BRAND_PRICE,
        "input": "Có điện thoại Samsung nào từ 5 đến 10 triệu không?",
        "filter": PhoneFilter(
            brand_code="samsung",
            min_price=5000000,
            max_price=10000000,
            config=Config(is_recommending=True)
        )
    },
    # Thêm các test case cơ bản khác
]

In [ ]:
def create_ground_truth(phones: list[PhoneModel], test_case_type: PhoneTestCaseType) -> str:
    if not phones:
        return "Xin lỗi, hiện tại không có điện thoại nào phù hợp với yêu cầu của bạn."
    
    if test_case_type == PhoneTestCaseType.NAME_ONLY:
        phone = phones[0]
        return f"""
        {phone.name} có giá {format_price(phone.price)}. 
        Thông số kỹ thuật chính:
        - Màn hình: {phone.screen}
        - Camera: {phone.camera}
        - Pin: {phone.battery}
        - Chip: {phone.chip}
        """
    
    if test_case_type in [PhoneTestCaseType.BRAND_PRICE, PhoneTestCaseType.PRICE_ONLY]:
        return f"""
        Có {len(phones)} điện thoại phù hợp với yêu cầu của bạn:
        {format_phone_list(phones)}
        """
    
    # Xử lý các trường hợp khác

In [ ]:
from uuid import uuid4
from models.user import UserRole
from repositories.user import create as create_user, CreateUserModel
from repositories.thread import create as create_thread, CreateThreadModel
from service.store_chatbot_v2 import  gen_answer

def get_actual_answer(input: str) -> str:
    user = create_user(CreateUserModel(user_name=str(uuid4()), role=UserRole.chainlit_user))
    thread = create_thread(CreateThreadModel(user_id=user.id, name=user.user_name))
    
    return gen_answer(
        thread_id=thread.id,
        history=[{"role": "user", "content": str(input)}],
        user_id=user.id,
    )
    